In [2]:
# Importar las bibliotecas necesarias
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

# Generar un par de claves (privada y pública)
def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )

# Cifrar un mensaje asimétrico
def cifrar_mensaje_asimetrico(mensaje, clave_publica):
    """Cifra un mensaje utilizando la clave pública."""
    mensaje_cifrado = clave_publica.encrypt(
        mensaje.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_cifrado

# Descifrar un mensaje asimétrico
def descifrar_mensaje_asimetrico(mensaje_cifrado, clave_privada):
    """Descifra un mensaje cifrado utilizando la clave privada."""
    mensaje_descifrado = clave_privada.decrypt(
        mensaje_cifrado,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_descifrado.decode()

# Validar la autenticidad del remitente
def validar_autenticidad(mensaje, firma, clave_publica):
    """Valida que el mensaje proviene del remitente verificando la firma con la clave pública."""
    try:
        clave_publica.verify(
            firma,
            mensaje.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False

# Firmar un mensaje
def firmar_mensaje(mensaje, clave_privada):
    """Genera una firma para el mensaje utilizando la clave privada."""
    firma = clave_privada.sign(
        mensaje.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return firma

# ==== USO ====

# 1. Generar claves si aún no se han creado (solo una vez)
generar_claves_asimetricas()

# 2. Cargar claves
clave_privada = cargar_clave_privada()
clave_publica = cargar_clave_publica()

# 3. Definir mensaje
mensaje = "Este es el mensaje que quiero firmar"

# 4. Firmar y validar
print("\n[Validación de Autenticidad]")
firma = firmar_mensaje(mensaje, clave_privada)
print(f"Firma generada: {firma}")

es_valido = validar_autenticidad(mensaje, firma, clave_publica)
print(f"¿El mensaje es auténtico? {'Sí' if es_valido else 'No'}")



Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.

[Validación de Autenticidad]
Firma generada: b'\x17)7PI\x9cS\xe5Q\x98\xdc\xd6L\xc6\xe4\xac\x8aW\x99\x90O\xf5\x97|x\x18A\xd5%\xbdk\xbed\xa0e\xd7\x1c\x91j\x8c\xcb\xd2\x06\xc3\xef\xd7\xf0C\xe9\xe8\xdf\x94\r\xff\r\xc2\x14[b\x0e"W\xaf@\x838\xe9\xef\x84\xa59\x97\xfbD\x00\x99\xc5\xaa~%e~0\x83\x8ay\x95\x03\xc7\x11\xbav\x87\x81J\xe6\xdf`\xa7\xcf5\xe3\xaax\x9d\x80\x17n\t\xfa;\x9c\xa3WI\x06}>\x92\x9e\x869\xf4\xc4\xbe\xed\x19\xb8\x84Te\x02\x80cQ\x9c.\xccX\xf4n\xeb:D\x90h\xef\x15&\xa6\xbf\xfb\x16\xf1I\x9d\xca\x88\xf5\xff\xcf\xe3@\xbb\x1c\xffjI\xe6\xc0\xc0\xca\xbe\x03\x07\x83\xe7\xef\xccG\xbeke~\x9f\x83\x8c\xcaM\x0f\xc0F\xb7\x0b\xb1\xd8#"\x1b\x03\x98GA\xd8I\xac\x0c]{\x03\x01\x89s\x81\xd8\xb5\xffs[\xc8\xac\x93sn_\xde2\x8eIWq@\xb1\xd2X\xa3A!6\\\xcc\x1a\xac\xbbQn"@\x87D\x9a\xc3\xb0b\x8b\xc9'
¿El mensaje es auténtico? Sí


¿Qué ventajas tiene el cifrado asimétrico?
las ventajas del cifrado asimetrico son:
no requiere compartir claves secretas previamente.
Autenticación y firma digital.
Confidencialidad y no repudio.
escabilidad.
componente clave de protocolos modernos.
A un que son muchas ventajas, el cifrado asimetrico suele ser mas lento que el simetrico. por eso, en la practica, se utiliza ambos combinados: asimetrico para intercambiar de forma segura una clave simetrica, y luego simetrico para  cifrar lo datos.

¿Qué posibles fallas de seguridad pueden existir?
Las posibles fallas de seguridad en el cifrado asimétrico pueden surgir tanto de vulnerabilidades técnicas como de errores humanos o de implementación.
las principales fallas son:
compartir una clave privada
uso de claves debiles o de tamaño insuficiente
ataques de intermediario
Errores en la implementacion del algoritmo
falsificacion de certificados digitales

¿Qué sucede si yo pierdo mi clave privada? ¿Qué sucede si la comparto?

Perder o compartir una clave privada en un sistema de cifrado asimétrico tiene consecuencias graves, ya que esa clave es el elemento central de la seguridad.

si pierdo mi clave privada  se pierde de forma definitiva el acceso a toda la información cifrada con la correspondiente clave pública, y no hay forma de recuperarla sin una copia de seguridad. Por otro lado, si la clave privada se comparte o se ve comprometida, cualquier persona que la posea puede suplantar la identidad del propietario, descifrar mensajes confidenciales y firmar documentos como si fuera él, lo que representa un riesgo crítico de seguridad.

Intente codificar la fábula de la lechera u otro texto largo ¿Por qué da un error? ¿Cómo se puede arreglar este error para compartir mensajes cifrados?

el cifrado asimetrico no esta diseñado para cifrar grandes volúmenes de datos directamente, si no que en realidad tiene un límite técnico de longitud que depende del tamaño de la clave.
con RSA de 2048 bits, solo puedes cifrar hasta 245 bytes de datos (menos si se usa relleno/padding como OAEP)
para solucionar el problema se utiliza un enfoque hibrido, que convina criptografia simetrica y asimetrica:
Generar una clave simétrica aleatoria (por ejemplo, AES de 256 bits).
Cifrar el mensaje completo (la fábula, en este caso) con esa clave usando un algoritmo simétrico como AES.
en efecto al cifrar textos largos con cifrado asimetrico ocurre el error porque este tipo de cifrado solo admite pequeñas cantidades de datos.
la solucion segura y eficiente es usar un sistema híbrido: cifrar el mensaje con un algoritmo simétrico rápido (como AES)

Comparta mensajes largos cifrados entre los estudiantes y que ellos validen la autencididad del remitente

para compartir mensajes largos con los estudiantes es necesario verificar la autenticidad del remitente, se necesita combinar cifrado simetrico para mas fluidez con criptografia asimetrica para el intercambio seguro y firma digital.